In [1]:
#python manage.py shell_plus --notebook
import os, sys
os.chdir(os.path.dirname(os.path.realpath('.')))
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [ ]:
from users.common import *
from users.models import *

In [9]:
from rest_framework.permissions import BasePermission
from django.contrib.auth.models import User
from django.contrib.auth import authenticate
from users.permissions import *
import sys
from users.models import *
from lims.models import *

class fundamental():
    def str_to_class(self,classname):
        return getattr(sys.modules[__name__], classname)
    def get_class_name(self,classobj):
        return classobj.__name__
    def get_model_id(self,classobj):
        class_name=self.get_class_name(classobj)
        return Module.objects.get(module=class_name).id
    def get_role_id(self,username):
        return ProfileRole.objects.get(user__user__username=username).role_id
    def get_role_name(self,username):
        return ProfileRole.objects.get(user__user__username=username).role.role
    def get_model_fields(self,model):
        return model._meta.fields
    def user_role_permission(self,username):
        try:
            profile_role = ProfileRole.objects.get(user__user__username=username)
            roles = {}
            role_perm=RolePermission.objects.filter(role=profile_role.role)
            model_perm = {}
            for permission_set in role_perm:
                module = Module.objects.get(id=permission_set.module_id).module
                model_perm[permission_set.module_id] = {}
                model_perm[permission_set.module_id]['module'] = module
                model_perm[permission_set.module_id]['create'] = permission_set.create
                model_perm[permission_set.module_id]['read'] = permission_set.read
                model_perm[permission_set.module_id]['update'] = permission_set.update
                model_perm[permission_set.module_id]['delete'] = permission_set.delete
                model_perm[permission_set.module_id]['type'] = permission_set.type
                model_perm[permission_set.module_id]['scope'] = permission_set.scope
            roles = {}
            roles['role_name']=fundamental().get_role_name(username)
            roles['permissions']=model_perm
            return roles
        except Exception as e:
            return e  
    def get_all_models(self):
        module_dict = {}
        for module in Module.objects.all():
            module_dict[module.module]=fundamental().str_to_class(module.module)
        return module_dict
    
    def skip_check(self,view,request):
        try:
            if (view.read in [True,False] and view.id) and str(request.method) == 'GET':
                return view.read
        except:
            pass
        try:
            if view.list in [True,False] and str(request.method) == 'GET':
                return view.list
        except:
            pass
        try:
            if view.create in [True,False] and str(request.method) == 'POST':
                return view.create
        except:
            pass
        try:
            if view.update in [True,False] and str(request.method) == 'PUT':
                return view.update
        except:
            pass
        try:
            if view.delete == True and str(request.method) == 'DELETE':
                return view.delete
        except:
            pass
        return None

In [ ]:
print(fundamental().get_model_id(Section))
module=Section
module_id=Module.objects.get(module=fundamental().get_class_name(module)).id
print(module_id)

In [ ]:
class get_query(object):
    def __init__(self,request,model):
        self.model=model
        self.model_name=fundamental().get_class_name(model)
        self.model_id=fundamental().get_model_id(model)
        self.method=request.method
        self.models=fundamental().get_all_models()
        if request.user.is_authenticated:
            self.role_id=fundamental().get_role_id(self.username)
            self.user=request.user
            self.username=request.user.username
            self.role_name=fundamental().get_role_name(self.username)

    def queryset(self):
        module=self.model
        module_id=fundamental().get_model_id(module)
        if not self.user.is_authenticated:
            return module.objects.none()
        username=self.username
        try:
            if fundamental().user_role_permission(username)['permissions'][module_id]['scope'] == None:
                filter_set=None
                value=None
                query_set=module.objects.all()
                print('all')
                pass
        except:
            try:
                #check scope.filter_set, scope.value
                scope=fundamental().user_role_permission(username)['permissions'][module_id]['scope']
                scope_instance=scope#Scope.objects.get(id=scope.id)
                filter_set=scope_instance.filter_set
                exec('auto_value='+str(scope_instance.value))
                if isinstance(auto_value, str):
                    filter_string=filter_set+'='+'"'+auto_value+'"'
                str1=fundamental().get_class_name(module)+'.objects.filter('+filter_string+')'
                str1='query_set='+str1
                print(str1)
                exec(str1)
            except:
                query_set=module.objects.none()
                print('None')
        return query_set

class CustomPermission():
    def has_object_permission(self,request,view):
        if fundamental().skip_check(view,request) in [True,False]:
            return fundamental().skip_check(view,request) in [True,False]
        try:
            username=request.user.username
            module=vew.model
            method=request.method
            module_id=fundamental().get_model_id(module)
            method_dict={'GET':'read','POST':'create','PUT':'update','DELETE':'delete'}
            method=method_dict[method]
            return fundamental().user_role_permission(username)['permissions'][module_id][method]
        except Exception as e:
            print(e)
            return False

    def has_object_permission(self, request, view, obj):
        try:
            pk=view.kwargs['pk']
            return queryset(request.user.username,view.model).filter(id=pk).count()>0
        except Exception as e:
            print(e)
            return False

In [ ]:
def get_queryset(self):
    print(self.model.__name__)
    return get_query(request.user.username,self.model,request.method).queryset()

In [ ]:
from django.contrib.auth.models import User
from django.contrib.auth import authenticate
from rest_framework.permissions import BasePermission
from users.filter import *

class CustomPermission(BasePermission):
    message = 'You do not have permission to perform this action.'
    def has_permission(self, request, view):
        try:
            if (view.read in [True,False] and view.id) and str(request.method) == 'GET':
                return view.read
        except:
            pass
        try:
            if view.list in [True,False] and str(request.method) == 'GET':
                return view.list
        except:
            pass
        try:
            if view.create in [True,False] and str(request.method) == 'POST':
                return view.create
        except:
            pass
        try:
            if view.update in [True,False] and str(request.method) == 'PUT':
                return view.update
        except:
            pass
        try:
            if view.delete == True and str(request.method) == 'DELETE':
                return view.delete
        except:
            pass
        try:
            username=str(request.user.username)
            self.username=username
            roles=fundamental().user_role_permission(username)
            model_id=fundamental().get_model_id(view.model)
            method=str(request.method)
            result=False
            if fundamental().get_model_id(view.model) in roles['permissions'].keys():
                self.message='nddd'
                if method=='GET':
                    result= roles['permissions'][model_id]['read']
                elif method=='POST':
                    result= roles['permissions'][model_id]['create']
                elif method=='PUT':
                    result= roles['permissions'][model_id]['update']
                elif method=='DELETE':
                    result= roles['permissions'][model_id]['delete']
                else:
                    result= False
            return result
        except Exception as e:
            self.message=str(e)
            return False
            


class AdminOnly(BasePermission):
    message='You are not authorized to access this page'
    def has_permission(self, request, view):
        if ProfileRole.objects.filter(user__user__username=request.user,role__role='Admin').count() >0:
            return True

class IsOwnerProfile(BasePermission):
    message='You are not authorized to update this data'
    def has_object_permission(self, request, view, obj):
        if not request.user.is_authenticated:
            return False
        if str(request.method) in view.IsOwner_method:
            return obj.user.username == request.user.username
        # Instance must have an attribute named `owner`.
        return True

